<a href="https://colab.research.google.com/github/gadmin7/EventExtraction/blob/main/eventExtraction_RAMS_conditionalRandomFields_Events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn_crfsuite
!pip install -q highlight_text

In [ ]:
import sklearn_crfsuite
import nltk
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from highlight_text import HighlightText, ax_text, fig_text
import random
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
data = []
with open(r"/content/drive/MyDrive/NLP_event_extraction/train.jsonlines",encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
data_test = []
with open(r"/content/drive/MyDrive/NLP_event_extraction/dev.jsonlines",encoding='utf-8') as f:
    for line in f:
        data_test.append(json.loads(line))

In [ ]:
df1 = pd.DataFrame(data_test)
df1.head(2)

,rel_triggers,gold_rel_links,doc_key,ent_spans,language_id,source_url,evt_triggers,split,sentences,gold_evt_links
0,[],[],nw_RC00e90a0209cf7c63e3faf5008f034002ef61cea93...,"[[28, 28, [['evt089arg02place', 1.0]]], [33, 3...",eng,http://www.huffingtonpost.com/james-waters/nav...,"[[40, 40, [['life.die.n/a', 1.0]]]]",dev,"[[Three, specific, points, illustrate, why, Am...","[[[40, 40], [33, 33], evt089arg01victim], [[40..."
1,[],[],nw_RC010f29c9438b939daa050b70d8127eb5b2b3cb927...,"[[64, 64, [['evt011arg02target', 1.0]]], [72, ...",eng,http://edition.cnn.com/2016/06/12/world/euro-2...,"[[65, 65, [['conflict.attack.selfdirectedbattl...",dev,"[[The, fixture, ,, anticipated, by, fans, as, ...","[[[65, 65], [64, 64], evt011arg01attacker], [[..."


In [ ]:
df = pd.DataFrame(data)
df.head(2)

,rel_triggers,gold_rel_links,doc_key,ent_spans,language_id,source_url,evt_triggers,split,sentences,gold_evt_links
0,[],[],nw_RC000462ebb18ca0b29222d5e557fa31072af8337e3...,"[[42, 43, [['evt090arg02victim', 1.0]]], [85, ...",eng,https://www.washingtonpost.com/news/powerpost/...,"[[69, 69, [['life.die.deathcausedbyviolenteven...",train,"[[Transportation, officials, are, urging, carp...","[[[69, 69], [85, 88], evt090arg01killer], [[69..."
1,[],[],nw_RC0006850e77f498cb669777ce71e4ebfc471911174...,"[[83, 85, [['evt105arg05origin', 1.0]]], [26, ...",eng,http://www.expressen.se/nyheter/womens-secret-...,"[[51, 51, [['movement.transportartifact.hide',...",train,"[[2014, :, ISIS, marches, into, Raqqa, and, ma...","[[[51, 51], [26, 30], evt105arg01transporter],..."


In [ ]:
df1.drop(['rel_triggers','gold_rel_links','language_id','source_url','split'],axis=1,inplace=True)

In [ ]:
df.drop(['rel_triggers','gold_rel_links','language_id','source_url','split'],axis=1,inplace=True)

In [ ]:
def add_pos_tags(sentences):
    tagged_sentences = []
    for sent in sentences:
        tagged_sent = nltk.pos_tag(sent)
        tagged_sentences.append(tagged_sent)
    return tagged_sentences

In [ ]:
def add_pos_tags_to_column(df, column_name):
    df[column_name + '_pos_tags'] = df[column_name].apply(add_pos_tags)
    return df

# Add POS tags to the 'Plot' column in both training and test DataFrames
df = add_pos_tags_to_column(df, 'sentences')
df1 = add_pos_tags_to_column(df1, 'sentences')

In [ ]:
%%time
dataframe_list = []
for idx in range(7329):

    row_tokens = []
    trigger_tags = []
    arg_tags = []
    doc_key = []
    sentence_num = []
    for i in range(len(df['sentences_pos_tags'][idx])):
        row_tokens.extend(df['sentences_pos_tags'][idx][i])
        for j in range(len(df['sentences_pos_tags'][idx][i])):
          sentence_num.append(str(i)+df['doc_key'][idx])

    for i in range(len(row_tokens)):
        trigger_tags.append('O')
        arg_tags.append('O')
        doc_key.append(df['doc_key'][idx])

    gold_link = df['gold_evt_links'][idx]

    for i in range(len(gold_link)):

        trigger_list = gold_link[i][0]
        arg_list = gold_link[i][1]
        arg_label = gold_link[i][2]

        trigger_beg = trigger_list[0]
        trigger_end = trigger_list[1]
        trigger_tags[trigger_beg] = 'B-'+ arg_label[:6]
        if trigger_beg != trigger_end:
            for trg in range(trigger_beg+1,trigger_end+1):
                trigger_tags[trg] = 'I-'+ arg_label[:6]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        arg_tags[arg_beg] = 'B-'+ arg_label[11:]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                arg_tags[arg] = 'I-'+ arg_label[11:]

    d1 = pd.DataFrame({'doc_key':doc_key,'token':row_tokens,'event_tags':trigger_tags,'arg_tags':arg_tags})
    dataframe_list.append(d1)

CPU times: user 18 s, sys: 140 ms, total: 18.1 s
Wall time: 19 s


In [ ]:
%%time
dataframe_test_list = []
for idx in range(df1.shape[0]):

    row_tokens = []
    trigger_tags = []
    arg_tags = []
    doc_key = []
    sentence_num = []
    for i in range(len(df1['sentences_pos_tags'][idx])):
        row_tokens.extend(df1['sentences_pos_tags'][idx][i])
        for j in range(len(df1['sentences_pos_tags'][idx][i])):
          sentence_num.append(str(i)+df1['doc_key'][idx])

    for i in range(len(row_tokens)):
        trigger_tags.append('O')
        arg_tags.append('O')
        doc_key.append(df1['doc_key'][idx])

    gold_link = df1['gold_evt_links'][idx]

    for i in range(len(gold_link)):

        trigger_list = gold_link[i][0]
        arg_list = gold_link[i][1]
        arg_label = gold_link[i][2]

        trigger_beg = trigger_list[0]
        trigger_end = trigger_list[1]
        trigger_tags[trigger_beg] = 'B-'+ arg_label[:6]
        if trigger_beg != trigger_end:
            for trg in range(trigger_beg+1,trigger_end+1):
                trigger_tags[trg] = 'I-'+ arg_label[:6]

        arg_beg = arg_list[0]
        arg_end = arg_list[1]
        arg_tags[arg_beg] = 'B-'+ arg_label[11:]
        if arg_beg != arg_end:
            for arg in range(arg_beg+1,arg_end+1):
                arg_tags[arg] = 'I-'+ arg_label[11:]

    d1 = pd.DataFrame({'doc_key':doc_key,'token':row_tokens,'event_tags':trigger_tags,'arg_tags':arg_tags})
    dataframe_test_list.append(d1)

CPU times: user 2.29 s, sys: 12.5 ms, total: 2.3 s
Wall time: 2.4 s


In [ ]:
data = pd.concat(dataframe_list)

In [ ]:
test = pd.concat(dataframe_test_list)

In [ ]:
def token_features(doc, i):
    token_with_pos = doc['token'][i]  # Get token with POS tag as a tuple
    token = token_with_pos[0]  # Extract the token itself
    pos_tag = token_with_pos[1]  # Extract the POS tag
    features = {
        'token.lower()': token.lower(),
        'token[-3:]': token[-3:],
        'token[-2:]': token[-2:],
        'token.isupper()': token.isupper(),
        'token.istitle()': token.istitle(),
        'token.isdigit()': token.isdigit(),
        'pos_tag': pos_tag,
        'pos_tag[:2]': pos_tag[:2],
    }
    if i > 0:
        prev_token_with_pos = doc['token'][i - 1]
        prev_token = prev_token_with_pos[0]
        prev_pos_tag = prev_token_with_pos[1]
        features.update({
            '-1:token.lower()': prev_token.lower(),
            '-1:token.istitle()': prev_token.istitle(),
            '-1:token.isupper()': prev_token.isupper(),
            '-1:pos_tag': prev_pos_tag,
            '-1:pos_tag[:2]': prev_pos_tag[:2],
        })
    else:
        features['BOS'] = True
    if i < len(doc) - 1:
        next_token_with_pos = doc['token'][i + 1]
        next_token = next_token_with_pos[0]
        next_pos_tag = next_token_with_pos[1]
        features.update({
            '+1:token.lower()': next_token.lower(),
            '+1:token.istitle()': next_token.istitle(),
            '+1:token.isupper()': next_token.isupper(),
            '+1:pos_tag': next_pos_tag,
            '+1:pos_tag[:2]': next_pos_tag[:2],
        })
    else:
        features['EOS'] = True
    return features

# Define function to extract features for each sentence
def sentence_features(doc):
    return [token_features(doc, i) for i in range(len(doc))]

# Define function to extract labels for each sentence
def sentence_labels(doc):
    return list(doc['event_tags'])

def prepare_data(df):
    X = [sentence_features(doc) for _, doc in df.groupby('doc_key')]
    y = [sentence_labels(doc) for _, doc in df.groupby('doc_key')]
    return X, y

In [ ]:
%%time
X_train, y_train = prepare_data(data)

CPU times: user 27.7 s, sys: 997 ms, total: 28.7 s
Wall time: 30.9 s


In [ ]:
X_test, y_test = prepare_data(test)

In [ ]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 3h 12min 26s, sys: 1min 16s, total: 3h 13min 42s
Wall time: 3h 14min 4s


AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

In [ ]:
predictions = crf.predict(X_test)

In [ ]:
from sklearn_crfsuite import metrics
metrics.flat_accuracy_score(y_test, predictions)

0.991998394662252

In [ ]:
metrics.sequence_accuracy_score(y_test, predictions)

0.07900432900432901

https://sklearn-crfsuite.readthedocs.io/en/latest/api.html#module-sklearn_crfsuite.metrics

In [ ]:
train_pred = crf.predict(X_train)

In [ ]:
metrics.sequence_accuracy_score(y_train, train_pred)

0.37044617273843633

In [ ]:
type(crf)

sklearn_crfsuite.estimator.CRF

In [ ]:
import pickle

# Save the CRF model to disk
with open('/content/drive/MyDrive/NLP_event_extraction/crf_model.pkl', 'wb') as f:
    pickle.dump(crf, f)

In [ ]:
# Load the CRF model from disk
with open('/content/drive/MyDrive/NLP_event_extraction/crf_model.pkl', 'rb') as f:
    loaded_crf_model = pickle.load(f)

In [ ]:
predictions = loaded_crf_model.predict(X_test)

In [ ]:
metrics.sequence_accuracy_score(y_test, predictions)

0.07900432900432901